In [ ]:
# Currently modifying model layers
# Validation Accuracy Goal: 70%

In [ ]:
# Loading training images, testing images, and image labels from CIFAR-10 folder

import os

training_path = 'CIFAR-10 Images/Training/'
testing_path = 'CIFAR-10 Images/Testing'
label_path = 'CIFAR-10 Images/batches.meta'

training_data = []
training_label_data = []
testing_data = []
testing_label_data = []

def unpickle_file(file):
    import pickle
    with open(file, 'rb') as tmp:
        dict = pickle.load(tmp, encoding='bytes')
    return dict

label_names = unpickle_file(label_path)[b'label_names']

def process_files(folder_path, data_container, label_container):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            dict = unpickle_file(file_path)
            data, labels = dict[b'data'], dict[b'labels']
            data_container.append(data)
            label_container.append(labels)

process_files(training_path, training_data, training_label_data)
process_files(testing_path, testing_data, testing_label_data)

In [ ]:
# Reshaping the image data from (3072,) to (32, 32, 3)

import numpy as np

def reshape_image(image_1d_array):
    # Setting up an empty 3D array of zeros to put in data
    image_reshaped = np.zeros((32, 32, 3), dtype=np.uint8)

    # Red
    image_reshaped[:, :, 0] = image_1d_array[:1024].reshape(32, 32)
    # Green
    image_reshaped[:, :, 1] = image_1d_array[1024:2048].reshape(32, 32)
    # Blue
    image_reshaped[:, :, 2] = image_1d_array[2048:].reshape(32, 32)

    return image_reshaped

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

def create_cnn_model():
    # model = models.Sequential([
    #     # layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    #     # Can be replace with these 2 lines but gives a warning
    #     # ============================================= #
    #     layers.Input(shape=(32, 32, 3)),
    #     layers.Conv2D(32, (3, 3), activation='relu'),
    #     # ============================================= #
    #     layers.MaxPooling2D((2, 2)),
    #     layers.Conv2D(64, (3, 3), activation='relu'),
    #     layers.MaxPooling2D((2, 2)),
    #     layers.Conv2D(64, (3, 3), activation='relu'),
    #     layers.Flatten(),
    #     layers.Dense(64, activation='relu'),
    #     layers.Dense(10)
    # ])
    
    model = models.Sequential([
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        layers.RandomContrast((0, 1)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ])
    
    # model = models.Sequential([
    #     layers.Input(shape=(32, 32, 3)),
    #     layers.Conv2D(16, (3, 3), activation='relu'),
    #     layers.MaxPooling2D((3, 3)),
    #     layers.Conv2D(32, (3, 3), activation='relu'),
    #     layers.Conv2D(64, (3, 3), activation='relu'),
    #     layers.Conv2D(128, (3, 3), activation='relu'),
    #     layers.Flatten(),
    #     layers.Dense(64, activation='relu'),
    #     layers.Dense(10)
    # ])

    model.compile(optimizer='adam',
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

    return model

In [ ]:
all_images = []
all_labels = []

for batch in training_data:
    for img in batch:
        reshaped_image = reshape_image(img)
        all_images.append(reshaped_image)

for batch in training_label_data:
    for label in batch:
        all_labels.append(label)

x_train = np.array(all_images)
y_train = np.array(all_labels)

x_val = np.array([reshape_image(img) for img in testing_data[0]])
y_val = np.array(testing_label_data[0])

model = create_cnn_model()
model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_val, y_val))

In [ ]:
# Just some code to display some of the pictures
# Not required so I've left it on the bottom

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

graph_size = 10

def show_images(images, labels, class_names, width):
    plt.figure(figsize=(graph_size, graph_size))
    for i in range(width * width):  # Display first 25 images
        plt.subplot(width, width, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i].reshape(32, 32, 3), cmap=plt.cm.binary)  # Reshape and display
        if width <= 8:
            plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
images, labels = training_data[0]
images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

show_images(images, labels, label_names, 5)